In [20]:
from tensorflow.keras.datasets   import boston_housing , mnist , fashion_mnist
from tensorflow.keras.models     import Sequential , clone_model , Model , load_model
from tensorflow.keras.layers     import Dense , Activation , InputLayer , Flatten , Input , BatchNormalization , Dropout , Embedding

# CNN
from tensorflow.keras.layers     import Conv2D , MaxPooling2D , AveragePooling2D

from tensorflow.keras            import optimizers  
from tensorflow.keras.callbacks  import EarlyStopping , ModelCheckpoint , Callback
from tensorflow.keras.optimizers import SGD , Adam , RMSprop

# 이미지 로드
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator 

# 자연어 처리
from tensorflow.keras.preprocessing.text          import Tokenizer
from tensorflow.keras.preprocessing.sequence      import pad_sequences

from keras.utils.np_utils      import to_categorical

from sklearn.datasets          import load_iris , load_breast_cancer , load_digits
from sklearn.model_selection   import train_test_split

import matplotlib.pyplot as plt

import tensorflow as tf
import numpy  as np
import pandas as pd


In [21]:
train_img_generator = ImageDataGenerator(rescale=1/255,
                                        rotation_range=40,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        fill_mode='nearest')
train_generator = train_img_generator.flow_from_directory(
    'C:/Users/crid2/ml-data/beertest04/train' , 
    target_size = (200, 200) , # 200->244
    batch_size  = 20 ,
    class_mode = 'categorical'
)

Found 5394 images belonging to 34 classes.


In [22]:
test_img_generator = ImageDataGenerator(rescale=1/255,
                                        rotation_range=40,
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        shear_range=0.2,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        fill_mode='nearest')
test_generator = test_img_generator.flow_from_directory(
    'C:/Users/crid2/ml-data/beertest04/test' , 
    target_size = (200, 200) , 
    batch_size  = 20 ,
    class_mode = 'categorical'
)

Found 224 images belonging to 34 classes.


In [23]:
def img_model() :
  img_model = Sequential()

  img_model.add( Conv2D(input_shape = (200, 200, 3) , filters = 16 , kernel_size = (3,3) , strides = (1,1) , padding = 'valid') )
  img_model.add( Activation('relu'))
  img_model.add( MaxPooling2D(pool_size = (2,2) , padding = 'valid' ) )
  
      
  img_model.add( Conv2D(filters = 32 , kernel_size = (3,3) , strides = (1,1) , padding = 'valid') )
  img_model.add( Activation('relu'))
  img_model.add( MaxPooling2D(pool_size = (2,2) , padding = 'valid' ) )
  

  img_model.add( Conv2D(filters = 64 , kernel_size = (3,3) , strides = (1,1) , padding = 'valid') )
  img_model.add( Activation('relu'))
  img_model.add( MaxPooling2D(pool_size = (2,2) , padding = 'valid' ) )
  

  img_model.add( Conv2D(filters = 64 , kernel_size = (3,3) , strides = (1,1) , padding = 'valid') )
  img_model.add( Activation('relu'))
  img_model.add( MaxPooling2D(pool_size = (2,2) , padding = 'valid' ) )
 

  img_model.add( Flatten() )
  img_model.add( Dense(units = 1024) )   # 1024 뉴런을 신규 추가했다. 특성을 좀더 여러개로 볼수있을것이다. 
  img_model.add( Activation('relu') )    # 드랍아웃을 0.5로 증가했다. 과연?
  img_model.add( Dropout(0.5) )
    
  img_model.add( Dense(units = 1024) )  
  img_model.add( Activation('relu') )

    
  
  img_model.add( Dense(units = 34) ) # 33->34 증가 
  img_model.add( Activation('softmax') )

  return img_model 


In [24]:
model = img_model()
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 198, 198, 16)      448       
                                                                 
 activation_21 (Activation)  (None, 198, 198, 16)      0         
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 99, 99, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 97, 97, 32)        4640      
                                                                 
 activation_22 (Activation)  (None, 97, 97, 32)        0         
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 48, 48, 32)       0         
 g2D)                                                 

In [25]:
model.compile(optimizer = Adam(lr = 0.001), 
                      loss= 'categorical_crossentropy' , 
                      metrics=['accuracy']
             ) 

In [26]:
model_history = model.fit(train_generator, validation_data=test_generator,epochs=25 ,verbose=1) 

Epoch 1/25
  7/270 [..............................] - ETA: 1:10 - loss: 3.5806 - accuracy: 0.0714

KeyboardInterrupt: 

In [ ]:
print('모델 저장')
model.save('beerwiki_testmodel_renew02.h5')

In [ ]:
import matplotlib.pyplot as plt

acc = model_history.history['accuracy']
val_acc = model_history.history['val_accuracy']
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'go', label='Training Loss')
plt.plot(epochs, val_loss, 'g', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [16]:
train_generator.class_indices

{'강서': 0,
 '강한IPA': 1,
 '경복궁': 2,
 '골목대장': 3,
 '곰표밀맥주': 4,
 '곰표섬머에일': 5,
 '광화문': 6,
 '남산': 7,
 '대한IPA': 8,
 '더프로미어오비': 9,
 '맥아더': 10,
 '상상페일에일': 11,
 '성산일출봉': 12,
 '아크': 13,
 '에일의정석': 14,
 '여수': 15,
 '오비': 16,
 '인생에일': 17,
 '제주백록담': 18,
 '제주위트에일': 19,
 '제주펠롱웨일': 20,
 '카스': 21,
 '카스라이트': 22,
 '카스제로': 23,
 '테라': 24,
 '퇴근길': 25,
 '하이트엑스트라콜드': 26,
 '하이트제로': 27,
 '하이트페일라거': 28,
 '한강': 29,
 '해운대': 30,
 '핸드앤몰트유미의위트에일': 31,
 '흑당밀키스타우트': 32,
 '흥청망청': 33}

In [40]:
img_src = 'c://users/crid2/downloads/test/01.jpg'
test_img = image.load_img(img_src, target_size=(244, 244))
x = image.img_to_array(test_img)
x = np.expand_dims(x, axis = 0)
image_ = np.vstack([x])
classes = model.predict(image_, batch_size = 10)
print('pred - ', classes[0])
np.argmax(classes[0])


pred -  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


10